# IRIS TensorFI

## Dependencies

-  Python 2.7

In [1]:
!pip install -q scipy scikit-learn PyYAML matplotlib keras==2.3.0 pandas typing tensorflow==1.14.0

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
%load_ext tensorboard

In [3]:
import csv
import logging
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.metrics import categorical_accuracy
from keras.models import load_model
from keras.objectives import categorical_crossentropy
from keras.utils import to_categorical

import TensorFI as ti

Using TensorFlow backend.


In [4]:
pd.set_option('display.max_rows', 5)

## Experiments configuration

In [5]:
SEED = 0
np.random.seed(SEED)

CWD_PATH = os.getcwd()
DATASET_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/iris-data-set.csv")
MODEL_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/iris_neural_network.h5")
CONFIG_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/config/default.yaml")
LOGS_PATH = os.path.join(CWD_PATH, "DNN-model/Iris/")

## Load Iris Dataset as categorical

In [6]:
# Load Iris dataset.
with open(DATASET_PATH, 'r') as csvfile:
    iris = list(csv.reader(csvfile))[1:]

num_input = 4
num_classes = 3

# The inputs are four floats: sepal length, sepal width, petal length, petal width.
inputs = np.array(iris)[:, :num_input].astype(
    np.float)  # We select the first three columns.

# Outputs are initially individual strings: setosa, versicolor or virginica.
outputs = np.array(iris)[:, 4]  # We select the 4th column.

# Convert the output strings to ints.
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

# Encode the category integers as binary categorical vairables.
outputs_cats = to_categorical(outputs_ints)

# Split the input and output data sets into training and test subsets.
inds = np.random.permutation(len(inputs))

train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test, outputs_test = inputs[test_inds], outputs_cats[test_inds]

## Configure Keras session to use Tensorflow Session

In [7]:
# TF-Keras Session
sess = tf.compat.v1.Session()
K.set_session(sess)

init = tf.compat.v1.global_variables_initializer()
sess.run(init)

## Load our Model

(Credits to Mathieu DUMONT)

Our model is :

- 4 features
- FC 5
- Activation Relu
- FC 3
- Activation Soft-Max
- 3 labels

In [8]:
# Model
K.set_learning_phase(0)
model = load_model(MODEL_PATH, custom_objects=None, compile=True)
model.summary()
x = model.input
y = model.output


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 25        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


## Evaluate the Model w/o FI

Evaluate:

In [9]:
# Evaluate model
with sess.as_default():
    y_preds = y.eval(feed_dict={x: inputs_test})
    accuracy = tf.reduce_mean(categorical_accuracy(outputs_test,
                                                   y_preds)).eval()
    loss = tf.reduce_mean(categorical_crossentropy(outputs_test,
                                                   y_preds)).eval()

Results:

In [10]:
"Loss/Accuracy (W/O FI) = " + str([loss, accuracy])

'Loss/Accuracy (W/O FI) = [0.18939355, 0.9866667]'

The predictions are (with the closest to 1.0 being the best choice):

In [11]:
pd.DataFrame(y_preds)  # Predictions

,0,1,2
0,9.915878e-01,0.008348,0.000064
1,1.703185e-06,0.305018,0.694980
...,...,...,...
73,2.961268e-08,0.203358,0.796642
74,9.914997e-01,0.008440,0.000061


## Evaluate the model with FI

### Arm the FI

In [12]:
fi = ti.TensorFI(sess,
                 configFileName=CONFIG_PATH,
                 logDir=LOGS_PATH,
                 logLevel=logging.DEBUG,
                 disableInjections=True,
                 name="IrisInjection",
                 fiPrefix="fi_")
fi.turnOnInjections()

DEBUG:root:Done setting logLevel to 10
INFO:root:Initializing the injector
INFO:root:Modifying graph in session 
INFO:root:Done modification of graph
INFO:root:Initializing the fault injection parameters
DEBUG:root:Initialized config file : FIConfig: {
	faultTypeScalar : None
	faultTypeTensor : bitFlip-element
	injectMap : {<Ops.ALL: 'ALL'>: 1.0}
	faultSeed : None
	skipCount : 0
 }
INFO:root:Initializing the fault log
INFO:root:Performing monkey patching
INFO:root:Done with init
INFO:root:Turning on injections


We can observe some warnings such as `WARNING:root:Operation LogSoftmax should be either added to `excludeOps` or should have his injection implemented.`.  This means that the operation is not supported by TensorFI and therefore has no fault injection capability. We have to implement FI for these operations:

-  `LogSoftmax`
-  `Sqrt`

In addition, TensorFI can explicitly ignore operations by adding them in the `excludeOps` function.

### Evaluate

Before evaluating, we will store the Tensorflow graph (where all operators are located) for debugging purposes.

In [13]:
# writer used to save graph. Use tensorboard to view.
writer = tf.compat.v1.summary.FileWriter(LOGS_PATH, sess.graph)

Evaluate :

In [14]:
with sess.as_default():
    y_preds = y.eval(feed_dict={x: inputs_test})

writer.close()
fi.turnOffInjections()

with sess.as_default():
    accuracy = tf.reduce_mean(categorical_accuracy(outputs_test,
                                                   y_preds)).eval()
    loss = tf.reduce_mean(categorical_crossentropy(outputs_test,
                                                   y_preds)).eval()

INFO:root:Calling newRun IrisInjection runCount = 1
INFO:root:Tensor("activation_2/Softmax:0", shape=(?, 3), dtype=float32) is not iterable
DEBUG:root:TensorList = [<tf.Tensor 'activation_2/Softmax:0' shape=(?, 3) dtype=float32>]
DEBUG:root:Calling oldRun on fi_activation_2/Softmax:0
DEBUG:root:Calling Operator MatMul  (  , [[4.6 3.1 1.5 0.2]
 [5.9 3.  5.1 1.8]
 [5.1 2.5 3.  1.1]
 [4.6 3.4 1.4 0.3]
 [6.2 2.2 4.5 1.5]
 [7.2 3.6 6.1 2.5]
 [5.7 2.9 4.2 1.3]
 [4.8 3.  1.4 0.1]
 [7.1 3.  5.9 2.1]
 [6.9 3.2 5.7 2.3]
 [6.5 3.  5.8 2.2]
 [6.4 2.8 5.6 2.1]
 [5.1 3.8 1.6 0.2]
 [4.8 3.4 1.6 0.2]
 [6.5 3.2 5.1 2. ]
 [6.7 3.3 5.7 2.1]
 [4.5 2.3 1.3 0.3]
 [6.2 3.4 5.4 2.3]
 [4.9 3.  1.4 0.2]
 [5.7 2.5 5.  2. ]
 [6.9 3.1 5.4 2.1]
 [4.4 3.2 1.3 0.2]
 [5.  3.6 1.4 0.2]
 [7.2 3.  5.8 1.6]
 [5.1 3.5 1.4 0.3]
 [4.4 3.  1.3 0.2]
 [5.4 3.9 1.7 0.4]
 [5.5 2.3 4.  1.3]
 [6.8 3.2 5.9 2.3]
 [7.6 3.  6.6 2.1]
 [5.1 3.5 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.2 3.4 1.4 0.2]
 [5.7 2.8 4.5 1.3]
 [6.6 3.  4.4 1.4]
 [5.  3.2

DEBUG:root:Calling Operator BiasAdd  (  , [[ 2.651197    5.678229   -0.3817312   3.8866897  -4.1688395 ]
 [ 1.4045125   7.714129    6.2685623   5.814866   -4.729314  ]
 [ 2.0396001   6.1032324   2.9344556   4.798796   -3.9524307 ]
 [ 2.8183255   5.816042   -0.3977946   3.9192405  -4.339905  ]
 [ 1.7385356   7.200938    5.076042    5.9636965  -4.299526  ]
 [ 1.8695581   9.278567    8.037237    7.247712   -5.6483345 ]
 [ 1.7381656   7.223717    4.4104257   5.397528   -4.5775075 ]
 [ 2.7955892   5.7227297  -0.7351266   4.0137725  -4.1932216 ]
 [ 1.6931415   8.802831    7.276508    6.996151   -5.259648  ]
 [ 1.8073912   8.685722    7.4148235   6.9110703  -5.245264  ]
 [ 1.4108231   8.343034    7.5704584   6.5301847  -4.9764047 ]
 [ 1.4113443   8.080285    7.239008    6.404739   -4.795639  ]
 [ 3.06187     6.5072355  -0.6188554   4.2787414  -4.867221  ]
 [ 2.7786536   6.0471535  -0.40792882  4.042807   -4.4691687 ]
 [ 1.8861281   8.217192    6.326752    6.417003   -5.0715847 ]
 [ 1.6488041 

DEBUG:root:	Returning from BiasAdd [[ 2.6242201e+00  5.9813404e+00 -6.2628639e-01  4.2499399e+00
  -4.1688395e+00]
 [ 1.3775358e+00  8.0172405e+00  6.0240073e+00  6.1781163e+00
  -4.7293139e+00]
 [ 2.0126233e+00  6.4063439e+00  2.6899004e+00  5.1620464e+00
  -3.9524307e+00]
 [ 2.7913487e+00  6.1191535e+00 -6.4234978e-01  4.2824907e+00
  -4.3399048e+00]
 [ 1.7115589e+00  7.5040498e+00  4.8314872e+00  6.3269467e+00
  -4.2995262e+00]
 [ 1.8425814e+00  9.5816784e+00  7.7926822e+00  7.6109624e+00
  -5.6483345e+00]
 [ 1.7111889e+00  7.5268288e+00  4.1658707e+00  5.7607784e+00
  -4.5775075e+00]
 [ 2.7686124e+00  6.0258412e+00 -9.7968179e-01  4.3770227e+00
  -4.1932216e+00]
 [ 1.6661648e+00  9.1059418e+00  7.0319529e+00  7.3594012e+00
  -5.2596478e+00]
 [ 1.7804145e+00  8.9888344e+00  7.1702685e+00  7.2743206e+00
  -5.2452641e+00]
 [ 1.3838464e+00  8.6461449e+00  7.3259034e+00  6.8934350e+00
  -4.9764047e+00]
 [ 1.3843676e+00  8.3833961e+00  6.9944530e+00  6.7679892e+00
  -4.7956390e+00]
 [ 3.

DEBUG:root:	Checking if operation Ops.RELU is chosen for injection
DEBUG:root:	Operation Ops.RELU is chosen for injection
DEBUG:root:	Perturbing [[ 2.6242201  5.9813404  0.         4.24994    0.       ]
 [ 1.3775358  8.017241   6.0240073  6.1781163  0.       ]
 [ 2.0126233  6.406344   2.6899004  5.1620464  0.       ]
 [ 2.7913487  6.1191535  0.         4.2824907  0.       ]
 [ 1.7115589  7.50405    4.831487   6.3269467  0.       ]
 [ 1.8425814  9.581678   7.792682   7.6109624  0.       ]
 [ 1.7111889  7.526829   4.1658707  5.7607784  0.       ]
 [ 2.7686124  6.025841   0.         4.3770227  0.       ]
 [ 1.6661648  9.105942   7.031953   7.359401   0.       ]
 [ 1.7804145  8.988834   7.1702685  7.2743206  0.       ]
 [ 1.3838464  8.646145   7.3259034  6.893435   0.       ]
 [ 1.3843676  8.383396   6.994453   6.767989   0.       ]
 [ 3.0348933  6.810347   0.         4.6419916  0.       ]
 [ 2.7516768  6.350265   0.         4.4060574  0.       ]
 [ 1.8591514  8.520304   6.082197   6.78025

DEBUG:root:Calling Operator MatMul  (  , [[   2.6242201    5.9813404    0.           4.24994      0.       ]
 [   1.3775358    8.017241     6.0240073    6.1781163    0.       ]
 [   2.0126233    6.406344     2.6899004    5.1620464    0.       ]
 [   2.7913487    6.1191535    0.           4.2824907    0.       ]
 [   1.7115589    7.50405      4.831487     6.3269467    0.       ]
 [   1.8425814    9.581678     7.792682     7.6109624    0.       ]
 [   1.7111889    7.526829     4.1658707    5.7607784    0.       ]
 [   2.7686124    6.025841     0.           4.3770227    0.       ]
 [   1.6661648    9.105942     7.031953     7.359401     0.       ]
 [   1.7804145    8.988834     7.1702685    7.2743206    0.       ]
 [   1.3838464    8.646145     7.3259034    6.893435     0.       ]
 [   1.3843676    8.383396     6.994453     6.767989     0.       ]
 [   3.0348933    6.810347     0.           4.6419916    0.       ]
 [   2.7516768    6.350265     0.           4.4060574    0.       ]
 [   1.

DEBUG:root:	Returning from MatMul [[ 8.55971813e+00  4.10511923e+00  1.64980143e-01]
 [-7.18058300e+00  5.23770905e+00  6.99368525e+00]
 [ 1.47010899e+00  4.49858952e+00  3.32042646e+00]
 [ 8.70727062e+00  4.22529221e+00  1.32181406e-01]
 [-3.29312801e+00  5.22685003e+00  5.83666134e+00]
 [-9.99247551e+00  6.49042892e+00  9.03724766e+00]
 [-2.06025028e+00  4.96034336e+00  4.93336105e+00]
 [ 8.81191254e+00  4.22936153e+00  1.84101880e-01]
 [-8.20473480e+00  6.14081335e+00  8.25102711e+00]
 [-8.73041725e+00  6.14987278e+00  8.37152481e+00]
 [-9.99655628e+00  5.76313210e+00  8.50831985e+00]
 [-9.22117233e+00  5.62897968e+00  8.15966988e+00]
 [ 9.48298931e+00  4.62300682e+00  1.12328917e-01]
 [ 8.92141628e+00  4.30295897e+00  1.42073840e-01]
 [-6.18023491e+00  5.80088949e+00  7.12208176e+00]
 [-8.31957531e+00  5.93856049e+00  7.99930096e+00]
 [ 8.37491798e+00  3.91928411e+00  2.91419774e-01]
 [-9.40502834e+00  5.72185564e+00  8.12345409e+00]
 [ 9.05505276e+00  4.33326674e+00  2.11435407e-0

DEBUG:root:	Returning from BiasAdd [[ 9.1874027e+00  4.4101386e+00 -4.6246597e-01]
 [-6.5528989e+00  5.5427284e+00  6.3662391e+00]
 [ 2.0977931e+00  4.8036089e+00  2.6929803e+00]
 [ 9.3349552e+00  4.5303116e+00 -4.9526471e-01]
 [-2.6654439e+00  5.5318694e+00  5.2092152e+00]
 [-9.3647909e+00  7.7954483e+00  8.4098015e+00]
 [-1.4325662e+00  5.2653627e+00  4.3059149e+00]
 [ 9.4395971e+00  4.5343809e+00 -4.4334424e-01]
 [-7.5770507e+00  6.4458327e+00  7.6235809e+00]
 [-8.1027327e+00  6.4548922e+00  7.7440786e+00]
 [-9.3688717e+00  6.0681515e+00  7.8808737e+00]
 [-8.5934877e+00  5.9339991e+00  7.5322237e+00]
 [ 1.0110674e+01  4.9280262e+00 -5.1511717e-01]
 [ 9.5491009e+00  4.6079783e+00 -4.8537228e-01]
 [-5.5525508e+00  6.1059089e+00  6.4946356e+00]
 [-7.6918912e+00  6.2435799e+00  7.3718548e+00]
 [ 9.0026026e+00  4.2243032e+00 -3.3602634e-01]
 [-8.7773438e+00  6.0268750e+00  7.4960079e+00]
 [ 9.6827374e+00  4.6382861e+00 -4.1601071e-01]
 [-8.0685854e+00  5.3931623e+00  6.8912282e+00]
 [-6.

DEBUG:root:	Returning from Softmax [[9.91587818e-01 8.34817812e-03 6.38919810e-05]
 [1.70318538e-06 3.05018455e-01 6.94979906e-01]
 [5.62433079e-02 8.41766655e-01 1.01989947e-01]
 [9.91822064e-01 8.12462904e-03 5.33609273e-05]
 [1.59694144e-04 5.79878390e-01 4.19961959e-01]
 [1.23820394e-08 3.51066828e-01 6.48933172e-01]
 [8.91013653e-04 7.22367048e-01 2.76741952e-01]
 [9.92596388e-01 7.35299662e-03 5.06600736e-05]
 [1.91359916e-07 2.35457271e-01 7.64542520e-01]
 [1.02834434e-07 2.15990528e-01 7.84009397e-01]
 [2.77251129e-08 1.40309438e-01 8.59690487e-01]
 [8.25459523e-08 1.68229893e-01 8.31770062e-01]
 [9.94394183e-01 5.58165880e-03 2.41454854e-05]
 [9.92860854e-01 7.09555112e-03 4.35483744e-05]
 [3.49301945e-06 4.04022455e-01 5.95974088e-01]
 [2.16842594e-07 2.44479567e-01 7.55520284e-01]
 [9.91573274e-01 8.33941903e-03 8.72187520e-05]
 [6.96023505e-08 1.87074423e-01 8.12925458e-01]
 [9.93555605e-01 6.40346203e-03 4.08659471e-05]
 [2.60261032e-07 1.82714105e-01 8.17285597e-01]
 [1.3

Results:

In [15]:
"Loss/Accuracy (FI) = " + str([loss, accuracy])

'Loss/Accuracy (FI) = [0.63154584, 0.94666666]'

The predictions are (with the closest to 1.0 being the best choice):

In [16]:
pd.DataFrame(y_preds)  # Predictions

,0,1,2
0,9.915878e-01,0.008348,0.000064
1,1.703185e-06,0.305018,0.694980
...,...,...,...
73,2.961268e-08,0.203358,0.796642
74,9.914997e-01,0.008440,0.000061


In [17]:
%tensorboard --logdir DNN-model/Iris/ --port 8008

Reusing TensorBoard on port 8008 (pid 16366), started 0:07:00 ago. (Use '!kill 16366' to kill it.)

In [18]:
sess.close()